In [208]:
import pandas as pd
import json
#import reportlab

In [209]:
path = 'noTimeCriticality/routing_output.json'
path2 = 'noTimeCriticality/sim_preprocessed_config.json'

routing_data_json = open(path)
routing_data_json = json.load(routing_data_json)

input_data_json = open(path2)
input_data_json = json.load(input_data_json)

In [210]:
# Extracting data from the JSON structure
routing_data = []
for day in routing_data_json["days"]:
    day_routes = []
    for vehicle in day["vehicles"]:
        day_routes.append(vehicle["route"])
    routing_data.append(day_routes)

# Creating a pandas DataFrame
df = pd.DataFrame(routing_data, columns=[f"Vehicle_{i+1}" for i in range(len(routing_data[0]))])
df = df.applymap(lambda x: [394 if elem == 395 else elem for elem in x])


In [211]:
df

,Vehicle_1,Vehicle_2,Vehicle_3,Vehicle_4,Vehicle_5,Vehicle_6,Vehicle_7,Vehicle_8,Vehicle_9
0,[],[],[],[],"[394, 129, 394]",[],[],[],"[394, 122, 394]"
1,[],[],[],[],[],[],[],[],[]
2,[],[],[],[],[],[],[],[],"[394, 103, 394]"
3,[],[],[],[],[],[],[],[],[]
4,[],[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...
246,[],[],[],[],"[394, 182, 394]",[],[],"[394, 45, 394]",[]
247,[],[],[],[],"[394, 182, 394]",[],[],[],"[394, 311, 394]"
248,[],[],[],[],[],[],"[394, 349, 394]",[],"[394, 178, 394]"
249,[],[],[],[],[],[],[],[],[]


In [212]:
# Extract pickup sites data
pickup_sites_data = [
    {
        "location_index": site["location_index"],
        "latitude": site["lonlats"][1],
        "longitude": site["lonlats"][0]
    }
    for site in input_data_json["pickup_sites"]
]

# Extract depots data
depots_data = [
    {
        "location_index": 394,
        "latitude": depot["lonlats"][1],
        "longitude": depot["lonlats"][0]
    }
    for depot in input_data_json["depots"]
]

# Create DataFrames
pickup_sites_df = pd.DataFrame(pickup_sites_data)
depots_df = pd.DataFrame(depots_data)

coord_df = pd.concat([pickup_sites_df, depots_df], ignore_index = True)

In [213]:
coord_df

,location_index,latitude,longitude
0,0,60.945884,23.352659
1,1,60.905353,23.329982
2,2,60.913446,23.435890
3,3,60.987171,23.410918
4,4,60.956071,23.491162
...,...,...,...
390,390,60.595201,23.277416
391,391,61.266603,23.503156
392,392,60.887524,22.669414
393,393,60.588183,23.446406


In [214]:
merged_df = df.applymap(lambda route: [coord_df.loc[idx, ['latitude', 'longitude']].tolist() for idx in route])
routes_with_dates = df.applymap(lambda route: [coord_df.loc[idx, ['latitude', 'longitude']].tolist() for idx in route])

In [215]:
routes_with_dates['Day'] = routes_with_dates.reset_index().index+1

In [216]:
merged_df

,Vehicle_1,Vehicle_2,Vehicle_3,Vehicle_4,Vehicle_5,Vehicle_6,Vehicle_7,Vehicle_8,Vehicle_9
0,[],[],[],[],"[[60.929833, 23.369694], [60.83979769075367, 2...",[],[],[],"[[60.929833, 23.369694], [61.036504853800366, ..."
1,[],[],[],[],[],[],[],[],[]
2,[],[],[],[],[],[],[],[],"[[60.929833, 23.369694], [60.99944150606829, 2..."
3,[],[],[],[],[],[],[],[],[]
4,[],[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...
246,[],[],[],[],"[[60.929833, 23.369694], [60.71122680641582, 2...",[],[],"[[60.929833, 23.369694], [60.89594067989363, 2...",[]
247,[],[],[],[],"[[60.929833, 23.369694], [60.71122680641582, 2...",[],[],[],"[[60.929833, 23.369694], [61.02120331743758, 2..."
248,[],[],[],[],[],[],"[[60.929833, 23.369694], [60.779703334185186, ...",[],"[[60.929833, 23.369694], [60.6542093588023, 23..."
249,[],[],[],[],[],[],[],[],[]


In [217]:
# Step 1: Create a boolean mask to identify rows with empty lists in all columns
empty_rows_mask = df.apply(lambda row: all(isinstance(cell, list) and len(cell) == 0 for cell in row), axis=1)

# Step 2: Use the boolean mask to get the indexes of rows with empty lists
empty_row_indexes = (empty_rows_mask[empty_rows_mask].index+1).tolist()

# Step 3: Count the number of empty rows
num_empty_rows = len(empty_row_indexes)

# Step 4: Print the indexes and count
print("No routes for days:", empty_row_indexes)
print("Days of no driving required:", num_empty_rows)

No routes for days: [2, 4, 5, 6, 8, 13, 22, 32, 42, 74, 115, 120, 127, 132, 142, 147, 158, 176, 188, 202, 212, 220, 222, 228, 231, 235, 240, 244, 250]
Days of no driving required: 29


In [220]:
routes_with_dates

,Vehicle_1,Vehicle_2,Vehicle_3,Vehicle_4,Vehicle_5,Vehicle_6,Vehicle_7,Vehicle_8,Vehicle_9,Day
0,[],[],[],[],"[[60.929833, 23.369694], [60.83979769075367, 2...",[],[],[],"[[60.929833, 23.369694], [61.036504853800366, ...",1
1,[],[],[],[],[],[],[],[],[],2
2,[],[],[],[],[],[],[],[],"[[60.929833, 23.369694], [60.99944150606829, 2...",3
3,[],[],[],[],[],[],[],[],[],4
4,[],[],[],[],[],[],[],[],[],5
...,...,...,...,...,...,...,...,...,...,...
246,[],[],[],[],"[[60.929833, 23.369694], [60.71122680641582, 2...",[],[],"[[60.929833, 23.369694], [60.89594067989363, 2...",[],247
247,[],[],[],[],"[[60.929833, 23.369694], [60.71122680641582, 2...",[],[],[],"[[60.929833, 23.369694], [61.02120331743758, 2...",248
248,[],[],[],[],[],[],"[[60.929833, 23.369694], [60.779703334185186, ...",[],"[[60.929833, 23.369694], [60.6542093588023, 23...",249
249,[],[],[],[],[],[],[],[],[],250


In [227]:
vehicle1 = routes_with_dates[['Vehicle_1', 'Day']]
vehicle2 = routes_with_dates[['Vehicle_2', 'Day']]
vehicle3 = routes_with_dates[['Vehicle_3', 'Day']]
vehicle4 = routes_with_dates[['Vehicle_4', 'Day']]
vehicle5 = routes_with_dates[['Vehicle_5', 'Day']]
vehicle6 = routes_with_dates[['Vehicle_6', 'Day']]
vehicle7 = routes_with_dates[['Vehicle_7', 'Day']]
vehicle8 = routes_with_dates[['Vehicle_8', 'Day']]
vehicle9 = routes_with_dates[['Vehicle_9', 'Day']]

In [236]:
mask1 = vehicle1['Vehicle_1'].apply(lambda x: len(x) == 0)
mask2 = vehicle2['Vehicle_2'].apply(lambda x: len(x) == 0)
mask3 = vehicle3['Vehicle_3'].apply(lambda x: len(x) == 0)
mask4 = vehicle4['Vehicle_4'].apply(lambda x: len(x) == 0)
mask5 = vehicle5['Vehicle_5'].apply(lambda x: len(x) == 0)
mask6 = vehicle6['Vehicle_6'].apply(lambda x: len(x) == 0)
mask7 = vehicle7['Vehicle_7'].apply(lambda x: len(x) == 0)
mask8 = vehicle8['Vehicle_8'].apply(lambda x: len(x) == 0)
mask9 = vehicle9['Vehicle_9'].apply(lambda x: len(x) == 0)

vehicle1 = vehicle1[~mask1]
vehicle2 = vehicle2[~mask2]
vehicle3 = vehicle3[~mask3]
vehicle4 = vehicle4[~mask4]
vehicle5 = vehicle5[~mask5]
vehicle6 = vehicle6[~mask6]
vehicle7 = vehicle7[~mask7]
vehicle8 = vehicle8[~mask8]
vehicle9 = vehicle9[~mask9]

In [240]:
vehicle1

,Vehicle_4,Day
6,"[[60.929833, 23.369694], [60.91501806870317, 2...",7
11,"[[60.929833, 23.369694], [60.90249027529243, 2...",12
24,"[[60.929833, 23.369694], [60.95210008052109, 2...",25
39,"[[60.929833, 23.369694], [60.856821059733406, ...",40
48,"[[60.929833, 23.369694], [60.93999766075358, 2...",49
51,"[[60.929833, 23.369694], [61.19748555126123, 2...",52
52,"[[60.929833, 23.369694], [60.998334609066326, ...",53
56,"[[60.929833, 23.369694], [60.806585946526894, ...",57
61,"[[60.929833, 23.369694], [60.859389833582014, ...",62
64,"[[60.929833, 23.369694], [61.12465303734291, 2...",65
